# Amazon Dataset

Datasets from: https://nijianmo.github.io/amazon/index.html#complete-data  
Take ratings only data (I downloaded ratings-only, magazine subscription as an example.)

In [40]:
import pandas as pd
import numpy as np
import scipy.io as sio
import h5py

In [41]:
# Mount drive with button (on the left vertical bar, under files)
# Copy file path

data = pd.read_csv('/content/drive/MyDrive/CS6101/Guan Hao/Sample Datasets/Magazine_Subscriptions.csv', names = ['item', 'user', 'rating', 'timestamp'])

In [42]:
data.head()

,item,user,rating,timestamp
0,B00005N7P0,AH2IFH762VY5U,5.0,1005177600
1,B00005N7P0,AOSFI0JEYU4XM,5.0,1004486400
2,B00005N7OJ,A3JPFWKS83R49V,3.0,1174694400
3,B00005N7OJ,A19FKU6JZQ2ECJ,5.0,1163116800
4,B00005N7P0,A25MDGOMZ2GALN,5.0,1405296000


In [43]:
data.shape

(89689, 4)

In [44]:
data = data.drop(['timestamp'], axis = 1).groupby(['item', 'user']).aggregate(np.mean).reset_index()

In [45]:
data['rating'] = [int(np.round(x)) for x in data['rating']]

In [46]:
set(data['rating'])

{1, 2, 3, 4, 5}

In [47]:
data.head()

,item,user,rating
0,B00005N7NQ,A10EYTJF60COV2,1
1,B00005N7NQ,A10L46FJZJLZ5E,4
2,B00005N7NQ,A10O0I4H323CND,4
3,B00005N7NQ,A12AZEN39APFWW,5
4,B00005N7NQ,A12R37H5BK72TY,2


In [48]:
data.shape

(88318, 3)

In [49]:
for category in ['item', 'user']:
  lst = list(set(data[category]))
  lst.sort()
  lst = [tup[::-1] for tup in list(enumerate(lst))]
  curr_dict = dict(lst)
  data[category] = [curr_dict[id] for id in data[category].values]

In [50]:
len(set(data['user']))

72098

In [51]:
88318 / (72098 * 2428)

0.0005045187670835911

In [52]:
len(set(data['item']))

2428

In [53]:
matrix_df = data.pivot_table(index = 'user', columns = 'item', values = 'rating').fillna(0).astype(int)

In [54]:
matrix_df.head()

item,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2388,2389,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399,2400,2401,2402,2403,2404,2405,2406,2407,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417,2418,2419,2420,2421,2422,2423,2424,2425,2426,2427
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
matrix_df.apply(lambda x: x > 0).sum(axis = 1).describe()

count    72098.000000
mean         1.224972
std          0.815326
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         55.000000
dtype: float64

In [56]:
matrix_df.apply(lambda x: x > 0).sum(axis = 0).describe()

count    2428.000000
mean       36.374794
std       115.470216
min         1.000000
25%         2.000000
50%         7.000000
75%        21.000000
max      1716.000000
dtype: float64

In [57]:
matrix_df.sum(axis = 0).describe()

count    2428.000000
mean      146.942339
std       471.101021
min         1.000000
25%        10.000000
50%        27.000000
75%        87.000000
max      7461.000000
dtype: float64

In [58]:
# set(np.ndarray.flatten(matrix_df.values))

In [59]:
matrix_df.shape

(72098, 2428)

# Flixster Example Mat File

In [77]:
# Reference: https://stackoverflow.com/questions/17316880/reading-v-7-3-mat-file-in-python
# No idea what the W matrices represent

dataset_name = 'douban' # (can switch to 'yahoo_music' and 'flixster')
filepath = f'/content/drive/MyDrive/CS6101/Guan Hao/Sample Datasets/training_test_dataset_{dataset_name}.mat'
example_data = {}
f = h5py.File(filepath)
for k, v in f.items():
    example_data[k] = np.array(v)
    print(k)
    print(example_data[k].shape)
    print(np.unique(example_data[k]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  if __name__ == '__main__':


M
(3000, 3000)
[0. 1. 2. 3. 4. 5.]
Otest
(3000, 3000)
[0. 1.]
Otraining
(3000, 3000)
[0. 1.]
W_users
(3000, 3000)
[0. 1.]


In [85]:
# Checks (Should have no output)

for i in range(3000):
  for j in range(3000):
    if example_data['Otest'][i][j] == 1 or example_data['Otraining'][i][j] == 1:
      if example_data['M'][i][j] == 0:
        print((i, j))

    if example_data['Otest'][i][j] == 0 and example_data['Otraining'][i][j] == 0:
      if example_data['M'][i][j] > 0:
        print((i, j))

In [ ]:
# Uncomment to see examples

# example_data['M']
# example_data['Otraining']
# example_data['Otest']
# example_data['W_users']

# Train Test Split for Amazon Dataset

In [97]:
matrix.shape

(72098, 2428)

In [102]:
np.random.seed(1)

threshold = 20 # testing split percentage (example)

# train_test_split
matrix = matrix_df.values
Otraining_matrix = matrix.copy()
Otest_matrix = matrix.copy()

for i in range(matrix.shape[0]):
  for j in range(matrix.shape[1]):
    if matrix[i][j] > 0:
      random_int = np.random.randint(0, 100)

      if random_int < threshold:
        Otest_matrix[i][j] = 1
        Otraining_matrix[i][j] = 0

      else:
        Otraining_matrix[i][j] = 1
        Otest_matrix[i][j] = 0

In [103]:
# Links: 
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.savemat.html
# https://stackoverflow.com/questions/35706697/saving-numpy-structure-array-to-mat-file

mdic = {"M": matrix, "Otraining": Otraining_matrix, "Otest": Otest_matrix}
sio.savemat('/content/drive/MyDrive/CS6101/Guan Hao/Sample Datasets/Magazine_Subscriptions.mat', mdic)